# Text data preprocessing

## 1. Intro:

#### Text data:

In [1]:
# When we have text data, it's not suitable for the machine learning algorithm
# So we need to convert it to numerical data

In [2]:
# Bag of Words : list of unique words in text corpus
# Corpus : The whole collection of words that we have
# Term frequency - Inverse document frequency (TF-IDF): count of the numebr of times each word appears

#### Tf-Idf Vectorizes:

In [3]:
# Term frequency (TF) : 
# (Num of items term 't' appear in a document) / (Number of terms in document)

In [4]:
# Inverse document frequency (IDF) :
# log(N/n), 'N' : number of words , 'n' : number of documents a term 't' has appeared in.

#### Coclusion:

In [5]:
# IDF value of RARE word is HIGH
# IDF value of FREQUENT word is LOW
# TF-IDF value = DF * IDF. it represents the numeric value of the word

## 2. Import libraries

In [26]:
import numpy as np
import pandas as pd
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression

#### Stop words:

In [2]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [3]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

## 3. Data import and first look

#### First look:

In [4]:
news_dataset = pd.read_csv('train.csv')

In [5]:
# Label : 1:Fake news, 0:Real news
news_dataset.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [6]:
# We have 20,800 rows, 5 columns
news_dataset.shape

(20800, 5)

In [7]:
# We have a good distribution
news_dataset.label.value_counts()

1    10413
0    10387
Name: label, dtype: int64

#### Null values:

In [8]:
# As we can see there are some missing values:
news_dataset.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [9]:
# Replacing missing values with null string:
news_dataset = news_dataset.fillna('')

## 4. Data prepareation

#### Combining words from 2 columns:

In [10]:
# In this case we will use only title, author columns
# So we need to combine these 2 columns by creating new column 'content'
news_dataset['content'] = news_dataset['author'] + ' ' + news_dataset['title']

In [11]:
news_dataset.head()

,id,title,author,text,label,content
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1,Darrell Lucus House Dem Aide: We Didn’t Even S...
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0,"Daniel J. Flynn FLYNN: Hillary Clinton, Big Wo..."
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1,Consortiumnews.com Why the Truth Might Get You...
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1,Jessica Purkiss 15 Civilians Killed In Single ...
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1,Howard Portnoy Iranian woman jailed for fictio...


#### Seperating features and label:

In [12]:
X = news_dataset.drop(columns = 'label', axis = 1)
Y = news_dataset['label']

#### Stemming: reducing a word to it's keyword

In [13]:
# Example : enjoyable, enjoyment, are all types of 'enjoy'

In [14]:
port_stem = PorterStemmer()

In [15]:
# Create stemming function:
def stemming (content) :
    # Include all the words between a-z(A-Z) without any signs or marks:
    stemmed_content = re.sub('[^a-zA-Z]',' ',content)
    # Convert all the words to lower-case letters:
    stemmed_content = stemmed_content.lower()
    # Seperate all the words:
    stemmed_content = stemmed_content.split()
    # Stemming process: apply stemming + drop stopwords:
    stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
    # Seperate the words with space ' ':
    stemmed_content = ' '.join(stemmed_content)
    return stemmed_content

In [16]:
news_dataset['content'] = news_dataset['content'].apply(stemming)

In [17]:
print (news_dataset['content'])

0        darrel lucu hous dem aid even see comey letter...
1        daniel j flynn flynn hillari clinton big woman...
2                   consortiumnew com truth might get fire
3        jessica purkiss civilian kill singl us airstri...
4        howard portnoy iranian woman jail fiction unpu...
                               ...                        
20795    jerom hudson rapper trump poster child white s...
20796    benjamin hoffman n f l playoff schedul matchup...
20797    michael j de la merc rachel abram maci said re...
20798    alex ansari nato russia hold parallel exercis ...
20799                            david swanson keep f aliv
Name: content, Length: 20800, dtype: object


## 5. Feature extraction

#### Seperate labels and features:

In [18]:
# Features:
X = news_dataset['content'].values
# Label:
Y = news_dataset['label'].values

In [19]:
X.shape

(20800,)

In [20]:
Y.shape

(20800,)

#### Converting textual data to feature vectors:

In [22]:
vectorizer = TfidfVectorizer()
vectorizer.fit(X)
X = vectorizer.transform(X)

In [23]:
# We converted the words to numeric values
print(X)

  (0, 15686)	0.28485063562728646
  (0, 13473)	0.2565896679337957
  (0, 8909)	0.3635963806326075
  (0, 8630)	0.29212514087043684
  (0, 7692)	0.24785219520671603
  (0, 7005)	0.21874169089359144
  (0, 4973)	0.233316966909351
  (0, 3792)	0.2705332480845492
  (0, 3600)	0.3598939188262559
  (0, 2959)	0.2468450128533713
  (0, 2483)	0.3676519686797209
  (0, 267)	0.27010124977708766
  (1, 16799)	0.30071745655510157
  (1, 6816)	0.1904660198296849
  (1, 5503)	0.7143299355715573
  (1, 3568)	0.26373768806048464
  (1, 2813)	0.19094574062359204
  (1, 2223)	0.3827320386859759
  (1, 1894)	0.15521974226349364
  (1, 1497)	0.2939891562094648
  (2, 15611)	0.41544962664721613
  (2, 9620)	0.49351492943649944
  (2, 5968)	0.3474613386728292
  (2, 5389)	0.3866530551182615
  (2, 3103)	0.46097489583229645
  :	:
  (20797, 13122)	0.2482526352197606
  (20797, 12344)	0.27263457663336677
  (20797, 12138)	0.24778257724396507
  (20797, 10306)	0.08038079000566466
  (20797, 9588)	0.174553480255222
  (20797, 9518)	0.295420

## 6. Split to train and test

In [25]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size = 0.2, stratify=Y, random_state=2)

## 7. ML model : Logistic Regression

#### Run the model:

In [30]:
from sklearn.linear_model import LogisticRegression

In [31]:
model = LogisticRegression()

In [32]:
model.fit(X_train, Y_train)

LogisticRegression()

#### Accuracy score - Training data:

In [33]:
from sklearn.metrics import accuracy_score

In [34]:
X_train_prediction = model.predict(X_train)
training_data_accuracy = accuracy_score(X_train_prediction, Y_train)

In [37]:
print('Accuracy score of trianing data : ', training_data_accuracy)

Accuracy score of trianing data :  0.9865985576923076


#### Accuracy score - Test data:

In [38]:
X_test_prediction = model.predict(X_test)
test_data_accuracy = accuracy_score(X_test_prediction, Y_test)

In [39]:
# It's days our model is accurate:
print('Accuracy score of trianing data : ', test_data_accuracy)

Accuracy score of trianing data :  0.9790865384615385


#### Make prediction on datapoint that we have:

In [40]:
# We can see it equals 1 so it's fake news
X_new = X_test[0]
prediction = model.predict(X_new)
print(prediction)

[1]


#### Check if it's correct:

In [41]:
# Y_test contains the label, and it's actually correct:
print(Y_test[0])

1
